## The CrossTree

Voting Schema with multiple binary classification trees. The network implements a voting scheme based on three different trees:

* Cultural **Agnostic-Rappresentative** tree
* Cultural **Agnostic-Exclusive** tree
* Cultural **Exclusive-Rappresentative** tree

the most voted class will be the predicted class.

### Training Phase

The training process is quite standard and straight-forward: given the n G_features we want to directly predict the associated class.

### Employment Phase

The training model will be inserted in a wider model called X and utilized as a function for the computation of the G_Factor

## Dataset

Load the dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from CU_Dataset_Factory import Hf_Loader, CU_Dataset_Factory

def onehot_encode(
    df_train: pd.DataFrame,
    df_test: pd.DataFrame,
    cat_cols: list[str]|None = None,
    num_cols: list[str]|None = None,
    sparse: bool = False
) -> tuple[pd.DataFrame, pd.DataFrame, OneHotEncoder]:
    
    """
    Applies One-Hot Encoding to df_train and df_test guaranteeing the same
    set of columns, even if train is missing categories who are in the test set.

    Parameters
    ----------
    df_train : pd.DataFrame
        Training DataFrame.
    df_test : pd.DataFrame
        Testing DataFrame.
    cat_cols : list[str], optional
        List of categorical columns to encode.
        If None, all columns of type 'object' are taken.
    num_cols : list[str], optional
        List of numerical (or non-categorical) columns to preserve.
        If None, all columns not in cat_cols are taken. 
    handle_unknown : str, default="ignore"
        Beahavior on unknown values in test (typically "ignore").
    sparse : bool, default=False
        If True, returns sparse matrix, otherwise dense.

    Returns
    -------
    df_train_enc : pd.DataFrame
        Training DataFrame with One-Hot Encoding + original num_cols.
    df_test_enc : pd.DataFrame
        Testing DataFrame with One-Hot Encoding + original num_cols.
    encoder : OneHotEncoder
        The fitted OneHotEncoder object, useful for future transform.
    """
    
    # 1) Identify category and numerical columns (if not given)
    if cat_cols is None:
        cat_cols = df_train.select_dtypes(include="object").columns.tolist()
    if num_cols is None:
        num_cols = [c for c in df_train.columns if c not in cat_cols]

    # 2) Fit encoder on all category data (train + test)
    all_cats = pd.concat([df_train[cat_cols], df_test[cat_cols]], 
                         axis=0, ignore_index=True)
    encoder = OneHotEncoder(
        sparse_output=sparse
    ).fit(all_cats)

    # 3) Transform separatly train and test
    X_train_ohe = encoder.transform(df_train[cat_cols])
    X_test_ohe  = encoder.transform(df_test[cat_cols])

    # 4) Name the new columns
    ohe_cols = encoder.get_feature_names_out(cat_cols).tolist()

    # 5) Compose the final DataFrames
    df_train_enc = pd.DataFrame(
        np.hstack([X_train_ohe.toarray() if sparse else X_train_ohe,
                   df_train[num_cols].values]), # type: ignore
        columns=ohe_cols + num_cols,
        index=df_train.index
    )
    df_test_enc = pd.DataFrame(
        np.hstack([X_test_ohe.toarray() if sparse else X_test_ohe,
                   df_test[num_cols].values]),
        columns=ohe_cols + num_cols,
        index=df_test.index
    )

    return df_train_enc[ohe_cols], df_test_enc[ohe_cols], encoder


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Produce the Dataset

In [ ]:
print('Cultural Dataset argumentation start')
factory = CU_Dataset_Factory('.')
train_l = Hf_Loader("sapienzanlp/nlp2025_hw1_cultural_dataset", 'train')
validation_l = Hf_Loader("sapienzanlp/nlp2025_hw1_cultural_dataset", 'validation')

factory.produce(train_l, 'train.tsv', ['category', 'subcategory','type','languages','reference','num_langs', 'ambiguos', 'G', 'n_mod', 'back_links'], 'label', 10, False)
factory.produce(validation_l, 'validation.tsv', ['category', 'subcategory','type','languages','reference','num_langs', 'ambiguos', 'G', 'n_mod', 'back_links'], 'label', 10, False)
print('End process')

In [44]:
train = pd.read_csv('train.tsv', sep='\t')
validation = pd.read_csv('validation.tsv', sep='\t')

In [45]:
train.head(5)

,Unnamed: 0,num_langs,reference,languages,type,subcategory,category,qid,wiki_name,label
0,0,6,5,2,entity,film,films,Q32786,916 (film),1
1,1,30,40,8,entity,musical group,music,Q371,!!!,2
2,2,4,1,3,entity,comics,comics and anime,Q3729947,¡Soborno!,2
3,3,38,67,9,entity,musical group,music,Q158611,+44 (band),2
4,4,2,7,1,entity,building,architecture,Q280375,"1 Monk Street, Monmouth",1


In [46]:
validation.head(5)

,Unnamed: 0,num_langs,reference,languages,type,subcategory,category,qid,wiki_name,label
0,0,61,40,10,entity,sports club,sports,Q15786,1. FC Nürnberg,2
1,1,7,4,4,entity,record label,music,Q268530,77 Records,1
2,2,68,166,10,entity,animated film,comics and anime,Q216153,A Bug's Life,2
3,3,12,3,7,entity,film,films,Q593,A Gang Story,1
4,4,60,91,10,entity,choreographer,performing arts,Q192185,Aaron Copland,2


In [47]:
y_train = train[['label']]
y_validation = validation[['label']]

id_train = train[['wiki_name']]
id_validation = validation[['wiki_name']]

fe_train = train[['languages', 'num_langs', 'reference']]
fe_validation = validation[['languages', 'num_langs', 'reference']]

fe_str_train = train[['category', 'subcategory', 'type']]
fe_str_validation = validation[['category', 'subcategory', 'type']]

In [48]:
train_cat, validation_cat, _ =  onehot_encode(fe_str_train, fe_str_validation, ['category'] )
train_scat, validation_scat, _ = onehot_encode(fe_str_train, fe_str_validation, ['subcategory'] )
train_t, validation_t, _ = onehot_encode(fe_str_train, fe_str_validation, ['type'] )

In [49]:
print(validation_cat.shape)
print(validation_scat.shape)
print(validation_t.shape)

(300, 19)
(300, 112)
(300, 2)


In [50]:
from sklearn.decomposition import PCA

In [51]:
# pca = PCA(n_components=10)
# validation_cat = pd.DataFrame(pca.fit_transform(validation_cat))
# train_cat = pd.DataFrame(pca.fit_transform(train_cat))
# pca = PCA(n_components=50)
# validation_scat = pd.DataFrame(pca.fit_transform(validation_scat))
# train_scat = pd.DataFrame(pca.fit_transform(train_scat))

In [52]:
print(validation_cat.shape)
print(validation_scat.shape)
print(validation_t.shape)

(300, 19)
(300, 112)
(300, 2)


In [53]:
train = pd.concat([fe_train, train_cat, train_scat, train_t, y_train], axis=1)
validation = pd.concat([fe_validation, validation_cat, validation_scat, validation_t, y_validation], axis=1) 

print(train.shape)

(6251, 137)


In [54]:
train.head(3)

,languages,num_langs,reference,category_architecture,category_biology,category_books,category_comics and anime,category_fashion,category_films,category_food,...,subcategory_traditional costume,subcategory_transport,subcategory_transport company,subcategory_tree,subcategory_visual arts,subcategory_writer,subcategory_writing style,type_concept,type_entity,label
0,2,6,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,8,30,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2
2,3,4,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2


In [55]:
validation.head(3)

,languages,num_langs,reference,category_architecture,category_biology,category_books,category_comics and anime,category_fashion,category_films,category_food,...,subcategory_traditional costume,subcategory_transport,subcategory_transport company,subcategory_tree,subcategory_visual arts,subcategory_writer,subcategory_writing style,type_concept,type_entity,label
0,10,61,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2
1,4,7,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
2,10,68,166,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2


## Network

In [56]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

### Agnostic-Representative classifier

In [57]:
# prepare the dataset in order to take only two classes and eliminates the labels of the elements
d = train.query("label == 0 or label == 1")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [58]:
ar_tree = MLPClassifier().fit(x, y)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [59]:
d = validation.query("label == 0 or label == 1")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [60]:
y_pred = ar_tree.predict(x)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.90      0.89       117
           1       0.84      0.83      0.83        76

    accuracy                           0.87       193
   macro avg       0.86      0.86      0.86       193
weighted avg       0.87      0.87      0.87       193



## Agnostic-Exclusive Classifier

In [61]:
# prepare the dataset in order to take only two classes and eliminates the labels of the elements
d = train.query("label == 0 or label == 2")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [62]:
ae_tree = MLPClassifier().fit(x, y)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [63]:
d = validation.query("label == 0 or label == 2")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [64]:
y_pred = ae_tree.predict(x)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.85      0.79       117
           2       0.80      0.67      0.73       107

    accuracy                           0.76       224
   macro avg       0.77      0.76      0.76       224
weighted avg       0.77      0.76      0.76       224



## Representative-Exclusive

In [65]:
# prepare the dataset in order to take only two classes and eliminates the labels of the elements
d = train.query("label == 1 or label == 2")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [66]:
from sklearn.ensemble import RandomForestClassifier

In [67]:
re_tree = RandomForestClassifier().fit(x, y)

In [68]:
d = validation.query("label == 1 or label == 2")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [69]:
y_pred = re_tree.predict(x)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           1       0.68      0.80      0.73        76
           2       0.84      0.73      0.78       107

    accuracy                           0.76       183
   macro avg       0.76      0.77      0.76       183
weighted avg       0.77      0.76      0.76       183



# Voting Schema


In [71]:
from scipy.stats import mode
from sklearn.base import ClassifierMixin, BaseEstimator
from sklearn.ensemble import IsolationForest

# Cultural-classification Network
class CABNet(BaseEstimator, ClassifierMixin):
    
    def __init__(self, ar_estimator, ae_estimator, re_estimator) -> None:
        self.ar = ar_estimator 
        self.ae = ae_estimator 
        self.re = re_estimator
        
    
    def __fit(self, e, X, y) -> BaseEstimator:
        return e.fit(X, y)


    def fit(self, X, y=None):
        d = X.query("label == 0 or label == 1 ")
        y = d['label'].astype(int).to_numpy()
        x = d.drop(['label'], axis=1).astype(float).to_numpy()
        self.ar = self.__fit(self.ar, x, y)
        d = X.query("label == 0 or label == 2")
        y = d['label'].astype(int).to_numpy()
        x = d.drop(['label'], axis=1).astype(float).to_numpy()
        self.ae = self.__fit(self.ae, x, y)
        d = X.query("label == 1 or label == 2")
        y = d['label'].astype(int).to_numpy()
        x = d.drop(['label'], axis=1).astype(float).to_numpy()
        self.re = self.__fit(self.re, x, y)
        return self
        
    def predict(self, X):
        v1 = self.ar.predict(X)
        v2 = self.ae.predict(X)
        v3 = self.re.predict(X)
  
        votes = np.vstack([v1, v2, v3])
        majority, _ = mode(votes, axis=0)
        return majority.ravel()

In [72]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, IsolationForest
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

In [73]:
model =  CABNet(RandomForestClassifier(n_estimators=100), RandomForestClassifier(n_estimators=500), RandomForestClassifier(n_estimators=500))

In [74]:
model = model.fit(train)

In [75]:
d = validation
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [76]:
y_pred = model.predict(x)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.86      0.80       117
           1       0.71      0.74      0.72        76
           2       0.70      0.57      0.63       107

    accuracy                           0.73       300
   macro avg       0.72      0.72      0.72       300
weighted avg       0.72      0.73      0.72       300

